In [15]:
import requests

import pymongo


client = pymongo.MongoClient(host="localhost",port=27017)
db = client['stock']
table_basic = db['basic']
table_quotes = db['quotes']

from lxml import etree
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}

response = requests.get(url='http://quote.eastmoney.com/stock_list.html',headers=headers)

html = etree.HTML(response.text.encode(response.encoding).decode('gbk'))

location = '上海'

print(len(html.xpath('//div[@id="quotesearch"]/ul/li/a/text()')))

for stock in html.xpath('//div[@id="quotesearch"]/ul/li/a/text()'):
    left = stock.index('(')
    name = stock[0:left]
    code = stock[left+1:-1]
    if name=='凌云B股':
        location = '深圳'
        print(name,code)
    data = {
        'location':location,
        'name':name,
        'code':code
    }
    # 上海和深圳可能有两个同名的股票
    # 上海股市也可能有同名股票，但是code不一样
    res = table_basic.update_one({'code':code}, {'$set': data}, True)
    # if res.modified_count>0:
    #     print(name,code)

4998
凌云B股 900957


In [16]:
# -*- coding: utf-8 -*-
# author:           inspurer(月小水长)
# pc_type           lenovo
# create_time:      2019/12/1 23:07
# file_name:        history_stock_quotes_scrapy.py
# github            https://github.com/inspurer
# qq邮箱            2391527690@qq.com
# 微信公众号         月小水长(ID: inspurer)

import requests

from lxml import etree

# from stock_analysis.utils import myDict
from utils import myDict

import pymongo

client = pymongo.MongoClient(host="localhost",port=27017)
db = client['stock']
table_basic = db['basic']

basis = table_basic.find()
print(basis.count())
stock_basis = myDict.AllowKeyRepeatDict()
for basic in basis:
    stock_basis.add(key=basic['name'],value=basic['code'])

stock_name = '中信证券'
stock_code = stock_basis.query(key=stock_name)[0]
table_quotes = db[stock_name+'_'+stock_code]


headers = {
    'Referer': 'http://quotes.money.163.com/trade/lsjysj_600030.html?year=2019&season=4',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}

base_url = 'http://quotes.money.163.com/trade/lsjysj_{code}.html'

4998


/Users/dxl/miniconda3/envs/Python_Scrapy/lib/python3.4/site-packages/ipykernel_launcher.py:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [17]:
# 2019 年
year = 2019
season = 1
# 四季度
for season in range(1,5):
    params = {
        'year':year,
        'season':season
    }
    response = requests.get(url=base_url.format(code=stock_code),headers=headers,params=params)

    html = etree.HTML(response.text)
    for tr in html.xpath('//table[@class="table_bg001 border_box limit_sale"]/tr'):
        tds = tr.xpath('.//td/text()')
        date = tds[0].replace('-','')
        open,high,low,close = tds[1],tds[2],tds[3],tds[4]
        change,change_pencentage = tds[5],tds[6]
        deal_amount,deal_money = tds[7].replace(',',''),tds[8].replace(',','') # 成交量，成交金额（万元）
        print(date,open,high,low,close,change,change_pencentage,deal_amount,deal_money)
        data = {
            'date': int(date),
            'open': open,
            'high': high,
            'low': low,
            'close': close,
            'change': change,
            'change_pencentage': change_pencentage,
            'deal_amount': deal_amount,
            'deal_money': deal_money
        }
        # 上海和深圳可能有两个同名的股票
        # 上海股市也可能有同名股票，但是code不一样
        table_quotes.update_one({'date': date}, {'$set': data}, True)



20190329 22.93 24.85 22.93 24.78 1.95 8.54 4233646 1018465
20190328 22.75 23.23 22.45 22.83 -0.14 -0.61 1676061 382593
20190327 23.12 23.27 22.63 22.97 0.06 0.26 1870655 429951
20190326 23.90 23.90 22.76 22.91 -0.79 -3.33 2572075 599102
20190325 24.30 24.54 23.61 23.70 -1.10 -4.44 2525337 608361
20190322 24.77 25.02 24.30 24.80 -0.10 -0.40 2043175 503897
20190321 24.65 25.44 24.42 24.90 0.34 1.38 3563240 890991
20190320 24.15 24.95 24.09 24.56 0.33 1.36 2588968 634245
20190319 24.65 24.75 24.09 24.23 -0.40 -1.62 2152355 525375
20190318 23.75 24.69 23.30 24.63 0.82 3.44 2994300 721053
20190315 23.61 24.10 23.43 23.81 0.44 1.88 2464579 586364
20190314 23.75 24.25 23.11 23.37 -0.55 -2.30 2833473 668172
20190313 24.73 24.77 23.86 23.92 -0.76 -3.08 2999075 725491
20190312 24.17 25.34 24.17 24.68 0.59 2.45 4310350 1066638
20190311 24.00 24.48 23.20 24.09 0.01 0.04 3791527 902473
20190308 25.00 25.52 24.08 24.08 -2.68 -10.01 6328067 1571374
20190307 26.38 27.88 25.95 26.76 0.06 0.22 6217520 1

20190730 23.20 23.64 23.16 23.41 0.24 1.04 1001973 235014
20190729 23.33 23.35 23.06 23.17 -0.22 -0.94 626721 145169
20190726 23.28 23.39 23.19 23.39 0.00 0.00 536303 124908
20190725 23.29 23.39 23.16 23.39 0.13 0.56 643832 149905
20190724 23.01 23.43 23.01 23.26 0.39 1.71 1086123 252902
20190723 22.83 23.06 22.76 22.87 0.07 0.31 501549 114781
20190722 23.29 23.35 22.78 22.80 -0.38 -1.64 923065 212152
20190719 22.84 23.35 22.83 23.18 0.46 2.02 1080637 249996
20190718 22.90 22.95 22.71 22.72 -0.37 -1.60 867816 198043
20190717 23.26 23.40 23.01 23.09 -0.34 -1.45 1206503 280056
20190716 23.61 23.76 23.43 23.43 -0.26 -1.10 780117 183770
20190715 23.10 24.06 22.93 23.69 0.32 1.37 1502423 353835
20190712 23.14 23.41 22.90 23.37 0.19 0.82 966705 225002
20190711 23.40 23.64 22.99 23.18 -0.02 -0.09 1237520 288583
20190710 23.34 23.40 23.04 23.20 -0.03 -0.13 786532 182524
20190709 23.35 23.44 22.92 23.23 0.06 0.26 957242 222034
20190708 23.48 23.65 22.90 23.17 -0.37 -1.57 1289454 300001
20190705